## Construct Training Data

### 1. Load Data

In [1]:
import random
import pandas as pd
from datasets import load_dataset

In [2]:
mnli = load_dataset('kakaobrain/kor_nli', 'multi_nli', trust_remote_code=True)

In [3]:
snli = load_dataset('kakaobrain/kor_nli', 'snli', trust_remote_code=True)

In [4]:
mnli['train'][0]

{'premise': '개념적으로 크림 스키밍은 제품과 지리라는 두 가지 기본 차원을 가지고 있다.',
 'hypothesis': '제품과 지리학은 크림 스키밍을 작동시키는 것이다.',
 'label': 1}

In [5]:
print(len(mnli['train']))
print(len(snli['train']))

392702
550152


In [6]:
dataset = mnli['train'].to_list() + snli['train'].to_list()

In [7]:
print(len(dataset))

942854


### 2. Construct Triplets

In [8]:
def construct_triplets(samples):
    entail_set, contra_set  = {}, {}
    for sample in samples:
        if sample['label'] == 0: # entailment
            entail_set[sample['premise']] = sample['hypothesis']
        elif sample['label'] == 2: # contradiction
            contra_set[sample['premise']] = sample['hypothesis']

    triplet_lst = []
    for premise in entail_set.keys():
        if premise in contra_set.keys():
            entail = entail_set[premise].replace('\"', '').replace('\'', '')
            contra = contra_set[premise].replace('\"', '').replace('\'', '')
            premise = premise.replace('\"', '').replace('\'', '') 
            if entail == 'n/a' or contra == 'n/a' or premise == 'n/a':
                print('None: ', entail, contra, premise)
            else:
                triplet = premise + '\t' + entail + '\t' + contra
                triplet_lst.append(triplet)
    return triplet_lst

In [9]:
triplets = construct_triplets(dataset)

None:  n/a n/a 날씨에 따라서든 그렇지 않든
None:  n/a n/a 토니 슈즈(그래서 클린턴은 슈즈와 양말을 갖게 될 것이다).
None:  n/a n/a 그래, 아니, 네가 아는 그 지역은 아니야. 네가 아는 TI 공장이야. 네가 아는 한, 그들이 무엇을 만들고, 날씨를 제외한 모든 것을.
None:  n/a n/a 웨스 위드마이어
None:  n/a n/a Vend?me Op?ra Madeleine
None:  뭔가 말도 안 돼 n/a 말도 안 되는 소리야.
None:  n/a n/a 생제르맹
None:  n/a n/a 불륜을 저지르지 마라!
None:  n/a n/a AICPA 미국 공인 회계사 APB 회계 원칙 이사회 ARB 회계 연구 게시판 COSO 트레드 웨이위원회 조직위원회 FAS 재무 회계 기준 FASB 재무 회계 기준위원회 FASAB 연방 회계 기준 자문위원회 FIN FASB Interpertation Form &amp; Content OMB Bulletin 97-01 , 1996 년 10 월 16 일 발행 GAO 일반 회계 사무소 NAA 전국 회계사 협회 OMB 관리 및 예산 사무국 SEC 증권 거래위원회 SFFAC 연방 재무 회계 개념 명세서 SFFAS 연방 재무 회계 표준 명세서 SOP 포지션 명세서
None:  n/a n/a 비엔나 소음 합창단
None:  n/a n/a 7. Soderstrom CA, Smith GS, Dischinger PC, McDuff DR, Hebel JR, Gorelick DA 등
None:  n/a n/a 원주민 내가 이기고, 나는 큰 법을 어기고...
None:  n/a n/a 우편요금은 개인용 컴퓨터를 소유하고 있는지 여부입니다.
None:  n/a n/a 국제통제연방정부(GAO/AIMD99)의 표준
None:  n/a n/a 보라색 공으로 점프하는 것은 정말 재미있어요!


In [10]:
triplets[:3]

['시즌 중에 알고 있는 거 알아? 네 레벨에서 다음 레벨로 잃어버리는 거야 브레이브스가 모팀을 떠올리기로 결정하면 브레이브스가 트리플 A에서 한 남자를 떠올리기로 결정하면 더블 A가 그를 대신하러 올라가고 A 한 명이 그를 대신하러 올라간다.\t사람들이 기억하면 다음 수준으로 물건을 잃는다.\t그들은 어떤 일에도 리콜을 하지 않는다.',
 '우리 번호 중 하나가 당신의 지시를 세밀하게 수행할 것이다.\t우리 팀의 일원이 당신의 명령을 엄청나게 정확하게 실행할 것이다.\t우리는 지금 아무도 자유롭지 않기 때문에 네가 직접 조치를 취해야 한다.',
 '어떻게 아세요? 이 모든 것이 다시 그들의 정보다.\t이 정보는 그들의 것이다.\t그들은 전혀 정보가 없다.']

In [11]:
print(len(triplets))

277811


### 3. Save to CSV

In [12]:
def save_to_csv(triplets, path):
    with open(path, 'w') as file:
        file.write('sent0\tsent1\thard_neg\n')
        file.write('\n'.join(triplets))

In [13]:
save_to_csv(triplets, 'kor_nli_triplets.csv')

In [14]:
train_df = pd.read_csv('kor_nli_triplets.csv', sep='\t')

In [15]:
train_df.head()

,sent0,sent1,hard_neg
0,시즌 중에 알고 있는 거 알아? 네 레벨에서 다음 레벨로 잃어버리는 거야 브레이브스...,사람들이 기억하면 다음 수준으로 물건을 잃는다.,그들은 어떤 일에도 리콜을 하지 않는다.
1,우리 번호 중 하나가 당신의 지시를 세밀하게 수행할 것이다.,우리 팀의 일원이 당신의 명령을 엄청나게 정확하게 실행할 것이다.,우리는 지금 아무도 자유롭지 않기 때문에 네가 직접 조치를 취해야 한다.
2,어떻게 아세요? 이 모든 것이 다시 그들의 정보다.,이 정보는 그들의 것이다.,그들은 전혀 정보가 없다.
3,내 워커가 고장나서 지금 화가 났어. 스테레오를 정말 크게 틀어야 해.,나는 내 워크맨이 고장나서 화가 나서 이제 스테레오를 정말 크게 틀어야 한다.,내 워크맨은 여전히 항상 그랬던 것처럼 잘 작동한다.
4,(슬레이트에 대한 읽기는 잭슨의 연구 결과를 받아들인다.),슬레이트는 잭슨의 발견에 대해 의견을 가지고 있었다.,슬레이트는 잭슨의 발견에 대해 아무런 의견도 갖고 있지 않았다.


In [16]:
null = train_df[train_df.isnull().any(axis=1)]

In [17]:
null['sent1'] 

Series([], Name: sent1, dtype: object)